In [17]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import dash
import plotly.express as px
from dash import dcc,html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

In [18]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
df = pd.read_csv(URL)
print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [19]:
df.head()

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California


In [20]:
df.columns

Index(['Date', 'Year', 'Month', 'Recession', 'Consumer_Confidence',
       'Seasonality_Weight', 'Price', 'Advertising_Expenditure', 'Competition',
       'GDP', 'Growth_Rate', 'unemployment_rate', 'Automobile_Sales',
       'Vehicle_Type', 'City'],
      dtype='object')

In [29]:
year_list=[]
for i in range(1980, 2024, 1):
    year_list.append(i)
#Create app
app = dash.Dash(__name__)
#Task 1 Add the Title to the Dashboard
app.layout = html.Div(children=[html.H1('Automobile Sales Statistics Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),
# TASK 2: Create a dropdown menu and add two Report options to it
     
#Dropdown to select Yearly Statistics οr Recession Period Statistics
    
    html.Div([id='output-container', className='chart-grid', style={'display': 'flex'},
                            html.H2('Select Statistics:', style={'margin-right': '2em'}),
                        dcc.Dropdown(id='dropdown-statistics', 
                   options=[
                           {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                           {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                           ],
                  placeholder='Select a report type',
                  style={'width':0.8,'padding':3,'font-size':20,'text-align-last':'center'}),
#Create a dropdown menu for selecting the YEAR options to it
                dcc.Dropdown(id='select-year', 
                   options=[{'label': i, 'value': i} for i in year_list],
                  placeholder='select-year',
                  style={'width':0.8,'padding':3,'font-size':20,'text-align-last':'center'})
                        
])
    #outer division ends

])
#layout ends
# Run the app
if __name__ == '__main__':
    app.run_server(port=8051)  # Use port 8051 instead of 8050

SyntaxError: invalid syntax (479954688.py, line 14)

In [42]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the dataset
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
df = pd.read_csv(URL)

# Preprocess the data
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
year_list = [i for i in range(1980, 2024, 1)]

# Initialize Dash App
app = dash.Dash(__name__)
app.title = "Automobile Sales Statistics Dashboard"

# Layout
app.layout = html.Div([
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'fontSize': 24}
    ),
    html.Div([
        html.Label("Select Report Type", style={'fontSize': 20}),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            placeholder="Select a report type",
            value="Select Statistics",
            style={'width': '80%', 'padding': '3px', 'fontSize': 20, 'textAlignLast': 'center'}
        )
    ], style={'marginBottom': '20px'}),
    html.Div([
        html.Label("Select Year", style={'fontSize': 20}),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder="Select-year",
            value="Select-year",
            style={'width': '80%', 'padding': '3px', 'fontSize': 20, 'textAlignLast': 'center'}
        )
    ], style={'marginBottom': '20px'}),
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
])

# Callback to enable/disable year dropdown
@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

# Callback to update output container
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'), Input('select-year', 'value')]
)
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter data for recession periods
        recession_data = df[df['Recession'] == 1]

        # Plot 1: Line chart - Automobile sales over recession periods
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales',
                           title="Automobile Sales During Recession Period")
        )

        # Plot 2: Bar chart - Average vehicles sold by type
        avg_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales',
                          title="Average Vehicles Sold by Vehicle Type During Recession")
        )

        # Plot 3: Pie chart - Expenditure share by vehicle type
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',
                          title="Total Expenditure Share by Vehicle Type")
        )

        # Plot 4: Bar chart - Effect of unemployment rate
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                          labels={'unemployment_rate': 'unemployment_rate', 'Automobile_Sales': 'Average Automobile Sales'},
                          title="Effect of Unemployment Rate on Vehicle Type and Sales")
        )

        return [
            html.Div(className='chart-item', children=[R_chart1, R_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[R_chart3, R_chart4], style={'display': 'flex'})
        ]

    elif input_year and selected_statistics == 'Yearly Statistics':
        # Filter data for selected year
        yearly_data = df[df['Year'] == int(input_year)]

        # Plot 1: Yearly Automobile sales
        yas = df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, x='Year', y='Automobile_Sales',
                           title="Yearly Automobile Sales")
        )

        # Plot 2: Monthly Automobile sales
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas, x='Month', y='Automobile_Sales',
                           title="Total Monthly Automobile Sales")
        )

        # Plot 3: Average vehicles sold by type
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',
                          title=f"Average Vehicles Sold by Vehicle Type in {input_year}")
        )

        # Plot 4: Advertisement expenditure by vehicle type
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type',
                          title="Total Advertisement Expenditure by Vehicle Type")
        )

        return [
            html.Div(className='chart-item', children=[Y_chart1, Y_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[Y_chart3, Y_chart4], style={'display': 'flex'})
        ]
    return html.Div("No data to display. Please select a report type and year.", style={'textAlign': 'center'})

# Run the Dash app
if __name__ == '__main__':
    app.run_server(port=8051)


In [40]:
yearly_data = df[df['Year'] ==2010]
exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
exp_data

,Vehicle_Type,Advertising_Expenditure
0,Mediumfamilycar,15151
1,Smallfamiliycar,5437
2,Sports,12416
3,Supperminicar,5135


In [47]:
#merge multiple .png images into one
from PIL import Image

# Load the images
image1 = Image.open(r"C:\Users\ΚΩΝΣΤΑΝΤΙΝΟΣ\Pictures\Final Data Vis Project\YearlyReportgraphs1.png")
image2 = Image.open(r"C:\Users\ΚΩΝΣΤΑΝΤΙΝΟΣ\Pictures\Final Data Vis Project\YearlyReportgraphs2.png")
image3 = Image.open(r"C:\Users\ΚΩΝΣΤΑΝΤΙΝΟΣ\Pictures\Final Data Vis Project\YearlyReportgraphs3.png")
image4 = Image.open(r"C:\Users\ΚΩΝΣΤΑΝΤΙΝΟΣ\Pictures\Final Data Vis Project\YearlyReportgraphs4.png")

# Get dimensions of the images (assuming all images are the same size)
width, height = image1.size

# Create a new image large enough to fit all 4 images
YearlyReportgraphs = Image.new("RGBA", (width * 2, height * 2))

# Paste the images into the merged image
YearlyReportgraphs.paste(image1, (0, 0))                # Top-left
YearlyReportgraphs.paste(image2, (width, 0))           # Top-right
YearlyReportgraphs.paste(image3, (0, height))          # Bottom-left
YearlyReportgraphs.paste(image4, (width, height))      # Bottom-right

# Save the merged image
YearlyReportgraphs.save("YearlyReportgraphs.png")

print("Merged image saved as 'YearlyReportgraphs.png'")


Merged image saved as 'YearlyReportgraphs.png'
